In [1]:
import pathlib
import requests
from prompt_toolkit.renderer import print_formatted_text

year = 2024
day = 12

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open("cookie.txt", "r") as file:
    session = file.read()

headers = {"cookie": f"session={session}"}

pathlib.Path("inputs").mkdir(parents=True, exist_ok=True)

with open(f"inputs/day_{str(day).zfill(2)}_input.txt", "wb") as file:
    file.write(requests.get(url, headers=headers).content)

In [257]:
lines = [x for x in open(f"inputs/day_{str(day).zfill(2)}_input.txt", "r")]
aoc = "".join(lines)
G = [[x for x in line.strip()] for line in lines]
R = len(G)
C = len(G[0])

print(lines[:5])
print(R, C)

['WEEEEEFFFFFFFFFFFTTTTTTTTTTOBBBBQGGGQQQQQQQQQQQQQQQEEGGGGGGGGGGGGGOEMMMMMMMMMMMCMMMIIIIINWWWWWWDDDDDDDDDDDOOOOOOOOOOOOOOOOOXXXXWWWWWWWWWWWWW\n', 'WWEEEEWWFFFFFFFFFFFNNNTTTTOOOBBBQQQGGQQQQQQQQQQQQQEEEEGGGGGGGGGGGGOOOMMMMMMMMMMMMMMIIIINNVNNWWWDDDDDDDDDOOOOOOOOOOOOOOOOOOOXXXXCQWWWWWWWWWWW\n', 'WWWEEEWFFFFFFFFFFFNNNNNNTTOOOOBOQQQQQQQQQQQQQQQQQQEEEEEEGGGGGGGGGXOOOMMMMMMMMMMMMMMMINNNNNNNWWWDDDDDDDDDVQOOOOOOOOOOOOOOOOOXXXXCCCWWWWWWWWWW\n', 'WWWWWWWFFFFFFFFFFFNNNNNNNTOOOOOOUUQQQQQQQQQQQQQQQQEEEEGGGGGGGGGGROOBBMMMMMMMMMMMMMMMINNNNNNNWWDDDDDDDDDDVVVOOOOOOOOOOOOOOOGXXXXCCWWWWWWWWWWW\n', 'WWWWWWWTWFFFFFFFFNNFNNJJNNOOOOOUUUUNNNQQQQQQQQQQEEEEEEEEEGGGGGRGRRRRBMMMMMMMMMMMMMMMNNNNNNNNNNDDDDDDDDVVVVVHOOOOOOOOOOOOOOGGXXXXXWWWWWWWWWWW\n']
140 140


In [ ]:
from collections import defaultdict

regions = [[None for _ in range(C)] for _ in range(R)]

region_list = defaultdict(list)

region_counter = 0


def grow(region_id, r, c):

    labelled = 0

    for d in [(0, 1), (0, -1), (-1, 0), (1, 0)]:
        rr = r + d[0]
        cc = c + d[1]
        if 0 <= rr < R and 0 <= cc < C:
            # print(regions[rr][cc], region_counter, regions[r][c])
            if regions[rr][cc] is None and G[r][c] == G[rr][cc]:
                regions[rr][cc] = region_id
                region_list[region_id].append((rr, cc))
                labelled += 1 + grow(region_id, rr, cc)
            else:
                pass

    return labelled


for r in range(R):
    for c in range(C):
        if regions[r][c] is None:
            regions[r][c] = region_counter
            region_list[region_counter].append((r, c))
            labelled = 1 + grow(region_counter, r, c)
        region_counter += 1


def print_reg():
    for y in regions:
        print("  ".join([str(f"{x:02d}") for x in y]))


# print_reg()


def calculate_perimeter(region):
    lines = defaultdict(int)
    for f in region:
        lines[(f[0] + 0.5, f[1])] += 1
        lines[(f[0] - 0.5, f[1])] += 1
        lines[(f[0], f[1] + 0.5)] += 1
        lines[(f[0], f[1] - 0.5)] += 1

    # if a border is in there twice, it's removed.

    return sum([1 for k, v in lines.items() if v == 1])


ans = 0
for id, region in region_list.items():
    area = len(region)
    perimeter = calculate_perimeter(region)
    ans += area * perimeter

print(ans)

In [261]:
from collections import defaultdict

#
# test = """RRRRIICCFF
# RRRRIICCCF
# VVRRRCCFFF
# VVRCCCJFFF
# VVVVCJJCFE
# VVIVCCJJEE
# VVIIICJJEE
# MIIIIIJJEE
# MIIISIJEEE
# MMMISSJEEE"""
#
# G = [[x for x in line.strip()] for line in test.split("\n")]
# R = len(G)
# C = len(G[0])


regions = [[None for _ in range(C)] for _ in range(R)]
region_list = defaultdict(list)
region_counter = 0


def grow(region_id, r, c):
    labelled = 0
    for d in [(0, 1), (0, -1), (-1, 0), (1, 0)]:
        rr = r + d[0]
        cc = c + d[1]
        if 0 <= rr < R and 0 <= cc < C:
            if regions[rr][cc] is None and G[r][c] == G[rr][cc]:
                regions[rr][cc] = region_id
                region_list[region_id].append((rr, cc))
                labelled += 1 + grow(region_id, rr, cc)
            else:
                pass

    return labelled


for r in range(R):
    for c in range(C):
        if regions[r][c] is None:
            regions[r][c] = region_counter
            region_list[region_counter].append((r, c))
            labelled = 1 + grow(region_counter, r, c)
        region_counter += 1


def print_reg():
    for y in regions:
        print("  ".join([str(f"{x:02d}") for x in y]))


# This was inspired by a hint.
# def calculate_perimeter(region):
#
#     corners = 0
#
#     for r in region:
#         for d in [(-1, -1), (1, -1), (-1, 1), (1, 1)]:
#             rx = (r[0] + d[0], r[1])
#             ry = (r[0], r[1] + d[1])
#             rxy = (r[0] + d[0], r[1] + d[1])
#
#             # corner means different in both dimensions.
#             # OR same in both dimensions but different diagonally.
#
#             if rx not in region and ry not in region:
#                 corners += 1
#             if rx in region and ry in region and rxy not in region:
#                 corners += 1
#
#     return corners


# This is closer to my original implementation but is wise to the issue of a line appearing to continue but being left or right.
# For each line, we follow it along until it changes.


def calculate_perimeter(region):
    lines = defaultdict(list)
    for f in region:
        lines[(f[0] + 0.5, f[1])].append(0)
        lines[(f[0] - 0.5, f[1])].append(1)
        lines[(f[0], f[1] + 0.5)].append(2)
        lines[(f[0], f[1] - 0.5)].append(3)

    lines = [(k, v[0]) for k, v in lines.items() if len(v) == 1]

    fence_count = 0

    for direction in [0, 1, 2, 3]:

        fences = [x[0] for x in lines if x[1] == direction]
        sorted_fences = sorted(
            fences, key=lambda x: (x[direction // 2], x[abs(direction // 2 - 1)])
        )

        curr_val = sorted_fences[0][direction // 2]
        curr_val_other = sorted_fences[0][abs(direction // 2 - 1)]
        fence_count += 1
        for fence in sorted_fences[1:]:
            next_val = fence[direction // 2]
            next_val_other = fence[abs(direction // 2 - 1)]
            if not (next_val == curr_val and next_val_other == curr_val_other + 1):
                fence_count += 1
                curr_val = next_val
            curr_val_other = next_val_other

    return fence_count


ans = 0
for id, region in region_list.items():
    area = len(region)
    perimeter = calculate_perimeter(region)
    ans += area * perimeter
    # print(f"{area=}, {perimeter=}")

# 1132578 too high.
# 778974 too low.
print(ans)

784982
